In [ ]:
from IPython.core.display_functions import clear_output
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
from IPython.display import display
import ipywidgets as widgets

movies_df = pd.read_csv('movies.csv')
credits_df = pd.read_csv('credits.csv')

ratings = movies_df.merge(credits_df, on='title', how='left')

model = NearestNeighbors(metric='cosine', algorithm='auto')

tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(ratings['overview'].fillna(''))
model.fit(tfidf_matrix)

def movie_recommend(movie_title, ratings, model, top_n=10):
    input_vector = tfidf.transform([movie_title])
    
    distances, indices = model.kneighbors(input_vector, n_neighbors=top_n+1)
    
    distances = distances.flatten()[1:]
    indices = indices.flatten()[1:]
    
    recommendations = ratings.iloc[indices]['title'].tolist()
    
    return recommendations[:5]

def user_interaction(movie_title):
    movies_recommended = movie_recommend(movie_title, ratings, model)
    if movies_recommended.empty:
        display('Sorry, no recommendations for that title.')
    else:
        display(movies_recommended)
        
input_widget = widgets.Text(
    placeholder='Enter a movie title...',
    description='Movie Title: ',
    disabled=False
)

button = widgets.Button(
    description='Get Recommendations',
    disabled=False,
    button_style='',
    tooltip='Get Recommendations',
    icon='check'
)

def button_clicked():
    clear_output()
    display(input_widget, button)
    user_interaction(input_widget.value)
    
input_widget.observe(input_widget, button_clicked)

display(input_widget, button)